In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
import itertools
import os
from datetime import datetime
from scipy import stats
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Disables all deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### A IDEIA AQUI É TENTAR PREENCHER OS VALORES NAN COM ALGORITMOS DE MACHINE LEARNING

Todas essas tentativas esbarraram no problema de como preencher os valores NaN por imputação uma vez que praticamente todas as minhas colunas tem valores NaN. Vou tatnar a bordagem de ir preenchendo aos poucos.

In [2]:
# Carregando os datasets para o treinamento e concantenando-os
df_1 = pd.read_csv("../data/train.csv")
df_2 = pd.read_csv("../data/training_extra.csv")
df = pd.concat([df_1, df_2], axis=0)
df = df.drop(columns='id')

# Transformar colunas booleanas em 0 e 1
df['Laptop Compartment'] = df['Laptop Compartment'].map({'Yes': 1, 'No': 0})
df['Waterproof'] = df['Waterproof'].map({'Yes': 1, 'No': 0})

# Preenchendo Weight Capacity (kg) com média
df['Weight Capacity (kg)'] = df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mean())

df.isna().sum()

Brand                   126758
Material                110962
Size                     87785
Compartments                 0
Laptop Compartment       98533
Waterproof               94324
Style                   104180
Color                   133617
Weight Capacity (kg)         0
Price                        0
dtype: int64

In [3]:
# Criando um exemplo de DataFrame
data = {
    'Brand': ['Nike', 'Puma', 'Adidas', 'Jansport', None, 'Nike', None],
    'Compartments': [3, 2, 5, 4, 3, 4, 2],
    'Weight Capacity (kg)': [10.0, 12.5, 8.0, 9.5, 10.0, 11.0, 12.0],
    'Price': [200.0, 150.0, 250.0, 180.0, 220.0, 210.0, 230.0]
}

df = pd.DataFrame(data)

# Pré-processamento: Lidar com NaN nas variáveis
df_cleaned = df.dropna(subset=['Brand'])  # Remover as linhas onde 'Brand' é NaN

# Label Encoding para a variável categórica 'Brand'
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_cleaned['Brand'] = label_encoder.fit_transform(df_cleaned['Brand'])

# Dividir as variáveis em X (features) e y (target)
X = df_cleaned[['Compartments', 'Weight Capacity (kg)', 'Price']]  # Variáveis independentes
y = df_cleaned['Brand']  # Target ('Brand')

# Treinar um RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X, y)

# Prever os valores faltantes de 'Brand'
df_missing = df[df['Brand'].isna()]  # Dados onde 'Brand' está faltando
X_missing = df_missing[['Compartments', 'Weight Capacity (kg)', 'Price']]  # Usando as mesmas variáveis

# Prever os valores de 'Brand' para as linhas faltantes
predicted_brand = rf_classifier.predict(X_missing)

# Imputando os valores previstos de volta no DataFrame original
df.loc[df['Brand'].isna(), 'Brand'] = label_encoder.inverse_transform(predicted_brand)

# Visualizar o DataFrame após imputação
print("DataFrame após imputação de NaNs em 'Brand':")
print(df)


DataFrame após imputação de NaNs em 'Brand':
      Brand  Compartments  Weight Capacity (kg)  Price
0      Nike             3                  10.0  200.0
1      Puma             2                  12.5  150.0
2    Adidas             5                   8.0  250.0
3  Jansport             4                   9.5  180.0
4      Nike             3                  10.0  220.0
5      Nike             4                  11.0  210.0
6      Nike             2                  12.0  230.0


C:\Users\julia\AppData\Local\Temp\ipykernel_20124\1342065373.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Brand'] = label_encoder.fit_transform(df_cleaned['Brand'])


In [4]:
# Criando um exemplo de DataFrame
data = {
    'Brand': ['Nike', 'Puma', 'Adidas', 'Jansport', 'Unknown', 'Nike', 'Unknown'],
    'Compartments': [3, 2, 5, 4, 3, 4, 2],
    'Weight Capacity (kg)': [10.0, 12.5, 8.0, 9.5, 10.0, 11.0, 12.0],
    'Price': [200.0, 150.0, 250.0, 180.0, 220.0, 210.0, 230.0]
}

df = pd.DataFrame(data)
print(df)

      Brand  Compartments  Weight Capacity (kg)  Price
0      Nike             3                  10.0  200.0
1      Puma             2                  12.5  150.0
2    Adidas             5                   8.0  250.0
3  Jansport             4                   9.5  180.0
4   Unknown             3                  10.0  220.0
5      Nike             4                  11.0  210.0
6   Unknown             2                  12.0  230.0


In [5]:
import pandas as pd

# Supondo que df seja o seu DataFrame
df_dummies = pd.get_dummies(df, columns=['Brand'], drop_first=False)

# Exibindo as primeiras linhas do DataFrame
print(df_dummies.head(10))


   Compartments  Weight Capacity (kg)  Price  Brand_Adidas  Brand_Jansport  \
0             3                  10.0  200.0         False           False   
1             2                  12.5  150.0         False           False   
2             5                   8.0  250.0          True           False   
3             4                   9.5  180.0         False            True   
4             3                  10.0  220.0         False           False   
5             4                  11.0  210.0         False           False   
6             2                  12.0  230.0         False           False   

   Brand_Nike  Brand_Puma  Brand_Unknown  
0        True       False          False  
1       False        True          False  
2       False       False          False  
3       False       False          False  
4       False       False           True  
5        True       False          False  
6       False       False           True  


In [6]:

# Separando os dados onde Brand_Unknown é 0 (treinamento)
train_data = df_dummies[df_dummies['Brand_Unknown'] == 0]

# Separando os dados onde Brand_Unknown é 1 (previsão)
predict_data = df_dummies[df_dummies['Brand_Unknown'] == 1]

print(train_data)
print("----------------------------------")
print(predict_data)

   Compartments  Weight Capacity (kg)  Price  Brand_Adidas  Brand_Jansport  \
0             3                  10.0  200.0         False           False   
1             2                  12.5  150.0         False           False   
2             5                   8.0  250.0          True           False   
3             4                   9.5  180.0         False            True   
5             4                  11.0  210.0         False           False   

   Brand_Nike  Brand_Puma  Brand_Unknown  
0        True       False          False  
1       False        True          False  
2       False       False          False  
3       False       False          False  
5        True       False          False  
----------------------------------
   Compartments  Weight Capacity (kg)  Price  Brand_Adidas  Brand_Jansport  \
4             3                  10.0  220.0         False           False   
6             2                  12.0  230.0         False           False   

   

In [7]:
# Separando os dados onde Brand_Unknown é 0 (dados de treino)
train_data = df_dummies[df_dummies['Brand_Unknown'] == 0]

# Separando os dados onde Brand_Unknown é 1 (dados de previsão)
predict_data = df_dummies[df_dummies['Brand_Unknown'] == 1]

# Variáveis numéricas para predição (features)
features = ['Compartments', 'Weight Capacity (kg)', 'Price']  # Substitua pelas suas colunas numéricas

# Colunas dummies de Brand como target (exceto 'Brand_Unknown')
target = [col for col in df_dummies.columns if 'Brand_' in col and col != 'Brand_Unknown']
print(target)

['Brand_Adidas', 'Brand_Jansport', 'Brand_Nike', 'Brand_Puma']


In [8]:
from sklearn.ensemble import RandomForestClassifier

# Separando as variáveis independentes (features) e a variável dependente (target)
X_train = train_data[features]  # Variáveis numéricas
y_train = train_data[target]    # Variáveis dummies (target)

# Treinando o RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [9]:
# Para os dados com valores ausentes em Brand (onde Brand_Unknown é 1)
X_predict = predict_data[features]  # Variáveis numéricas para previsão
print(X_predict)


   Compartments  Weight Capacity (kg)  Price
4             3                  10.0  220.0
6             2                  12.0  230.0


In [10]:
# Prevendo as colunas dummies de Brand (as categorias dummies)
predictions = model.predict(X_predict)
print(predictions)

[[False False  True False]
 [False False False False]]


In [11]:

# Convertendo as previsões de volta para a coluna "Brand" original (escolher a coluna com valor 1)
predicted_brands = pd.DataFrame(predictions, columns=target)

# Escolher a coluna com valor 1 para cada linha (a categoria prevista)
df_dummies.loc[df_dummies['Brand_Unknown'] == 1, 'Brand'] = predicted_brands.idxmax(axis=1)

# Verificando se todos os NaNs foram preenchidos
print(df_dummies['Brand'].isna().sum())  # Deve retornar 0

7


In [12]:
print(df_dummies)

   Compartments  Weight Capacity (kg)  Price  Brand_Adidas  Brand_Jansport  \
0             3                  10.0  200.0         False           False   
1             2                  12.5  150.0         False           False   
2             5                   8.0  250.0          True           False   
3             4                   9.5  180.0         False            True   
4             3                  10.0  220.0         False           False   
5             4                  11.0  210.0         False           False   
6             2                  12.0  230.0         False           False   

   Brand_Nike  Brand_Puma  Brand_Unknown Brand  
0        True       False          False   NaN  
1       False        True          False   NaN  
2       False       False          False   NaN  
3       False       False          False   NaN  
4       False       False           True   NaN  
5        True       False          False   NaN  
6       False       False       

In [13]:
import pandas as pd

# Exemplo de dataset original
data = {
    'Brand': ['Nike', 'Puma', 'Adidas', 'Jansport', 'Unknown', 'Nike', 'Unknown'],
    'Compartments': [3, 2, 5, 4, 3, 4, 2],
    'Weight Capacity (kg)': [10.0, 12.5, 8.0, 9.5, 10.0, 11.0, 12.0],
    'Price': [200.0, 150.0, 250.0, 180.0, 220.0, 210.0, 230.0]
}

df = pd.DataFrame(data)

# Aplicando get_dummies e removendo a coluna original 'Brand'
df_dummies = pd.get_dummies(df, columns=['Brand'], drop_first=False)

# Exibindo o DataFrame com as variáveis dummies
print(df_dummies)



   Compartments  Weight Capacity (kg)  Price  Brand_Adidas  Brand_Jansport  \
0             3                  10.0  200.0         False           False   
1             2                  12.5  150.0         False           False   
2             5                   8.0  250.0          True           False   
3             4                   9.5  180.0         False            True   
4             3                  10.0  220.0         False           False   
5             4                  11.0  210.0         False           False   
6             2                  12.0  230.0         False           False   

   Brand_Nike  Brand_Puma  Brand_Unknown  
0        True       False          False  
1       False        True          False  
2       False       False          False  
3       False       False          False  
4       False       False           True  
5        True       False          False  
6       False       False           True  


In [14]:
# Separando os dados de treino (onde Brand_Unknown é 0)
train_data = df_dummies[df_dummies['Brand_Unknown'] == False]

# Separando os dados de previsão (onde Brand_Unknown é 1)
predict_data = df_dummies[df_dummies['Brand_Unknown'] == True]

# Variáveis independentes (features) são as numéricas
features = ['Compartments', 'Weight Capacity (kg)', 'Price']

# Variáveis dependentes (target) são as dummies de Brand (exceto 'Brand_Unknown')
target = [col for col in df_dummies.columns if 'Brand_' in col and col != 'Brand_Unknown']

# X_train e y_train para o RandomForest
X_train = train_data[features]
y_train = train_data[target]


In [15]:
from sklearn.ensemble import RandomForestClassifier

# Inicializando o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinando o modelo com os dados de treino
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [16]:
# Para os dados com valores ausentes em Brand (onde Brand_Unknown é 1)
X_predict = predict_data[features]

# Prevendo as colunas dummies de Brand (as categorias dummies)
predictions = model.predict(X_predict)

# Convertendo as previsões de volta para as colunas de Brand
predicted_brands = pd.DataFrame(predictions, columns=target)

# Preencher as linhas de 'Brand' onde Brand_Unknown é True com as previsões
df_dummies.loc[df_dummies['Brand_Unknown'] == True, 'Brand'] = predicted_brands.idxmax(axis=1)

# Garantir que as linhas restantes de 'Brand' (onde Brand_Unknown é False) mantêm os valores originais
df_dummies['Brand'] = df_dummies['Brand'].fillna(df_dummies['Brand'].where(df_dummies['Brand_Unknown'] == False))


# Verificando se todos os NaNs foram preenchidos
print(df_dummies['Brand'].isna().sum())  # Deve retornar 0


7


In [17]:
print(predictions)

[[False False  True False]
 [False False False False]]


In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Exemplo de DataFrame com uma variável categórica 'target_column' com valores ausentes
df = pd.DataFrame({
    'feature_1': [1, 2, 3, 4, 5, 6, 7],  # Coluna numérica
    'feature_2': [10, 11, 12, 13, 14, 15, 16],  # Coluna numérica
    'target_column': ['A', 'B', None, 'A', 'B', None, 'A']  # Target com valores ausentes
})

# Dividir o dataset em duas partes:
# - Linhas com valores ausentes na coluna target
# - Linhas com valores preenchidos
df_with_target = df[df['target_column'].notna()]  # Linhas com valores no target
df_missing_target = df[df['target_column'].isna()]  # Linhas com valores ausentes no target

# Separar as variáveis preditoras (X) e o target (y)
X_train = df_with_target[['feature_1', 'feature_2']]  # Variáveis preditoras
y_train = df_with_target['target_column']  # Target (coluna categórica)

X_test = df_missing_target[['feature_1', 'feature_2']]  # Variáveis preditoras para as linhas com 'target_column' ausente


model = RandomForestClassifier()
# Treinar o modelo
model.fit(X_train, y_train)

# Prever os valores ausentes
predicted_values = model.predict(X_test)

# Preencher os valores ausentes no target
df.loc[df['target_column'].isna(), 'target_column'] = predicted_values

# Exibir o DataFrame preenchido
print(df)



   feature_1  feature_2 target_column
0          1         10             A
1          2         11             B
2          3         12             B
3          4         13             A
4          5         14             B
5          6         15             B
6          7         16             A


### COMEÇANDO TUDO DE NOVO

In [29]:
# Carregando os datasets para o treinamento e concantenando-os
df = pd.read_csv("../data/train.csv")
df =df.drop(columns='id')
# Preenchendo Weight Capacity (kg) com média
df['Weight Capacity (kg)'] = df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mean())
df.isna().sum()

Brand                   9705
Material                8347
Size                    6595
Compartments               0
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)       0
Price                      0
dtype: int64

In [20]:
df_with_target = df[df['Brand'].notna()]
df_with_target

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
299995,Adidas,Leather,Small,9.0,No,No,Tote,Blue,12.730812,129.99749
299996,Jansport,Leather,Large,6.0,No,Yes,Tote,Blue,26.633182,19.85819
299997,Puma,Canvas,Large,9.0,Yes,Yes,Backpack,Pink,11.898250,111.41364
299998,Adidas,Nylon,Small,1.0,No,Yes,Tote,Pink,6.175738,115.89080


In [21]:
df_with_target_without_nan = df_with_target.dropna(axis=1, how='any')
df_with_target_without_nan

,Brand,Compartments,Weight Capacity (kg),Price
0,Jansport,7.0,11.611723,112.15875
1,Jansport,10.0,27.078537,68.88056
2,Under Armour,2.0,16.643760,39.17320
3,Nike,8.0,12.937220,80.60793
4,Adidas,1.0,17.749338,86.02312
...,...,...,...,...
299995,Adidas,9.0,12.730812,129.99749
299996,Jansport,6.0,26.633182,19.85819
299997,Puma,9.0,11.898250,111.41364
299998,Adidas,1.0,6.175738,115.89080


In [157]:
# Carregando os datasets para o treinamento e concantenando-os
df = pd.read_csv("../data/train.csv")
df =df.drop(columns='id')
# Preenchendo Weight Capacity (kg) com média
df['Weight Capacity (kg)'] = df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mean())
df.isna().sum()

Brand                   9705
Material                8347
Size                    6595
Compartments               0
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)       0
Price                      0
dtype: int64

In [158]:
# Selecionar as colunas do tipo 'object'
categorical_columns = df.select_dtypes(include=['object'])

# Filtrar as colunas que têm valores faltantes (NaN)
columns_with_nan = categorical_columns.columns[categorical_columns.isna().any()]

# Criar uma lista com o nome das colunas e o número de valores ausentes
columns_nan_count = [(column, df[column].isna().sum()) for column in columns_with_nan]

# Ordenar a lista pelo número de valores ausentes (de forma crescente)
columns_nan_count_sorted = sorted(columns_nan_count, key=lambda x: x[1])

# Extrair somente os nomes das colunas, agora ordenados
sorted_columns = [column for column, _ in columns_nan_count_sorted]
print(sorted_columns)


['Size', 'Waterproof', 'Laptop Compartment', 'Style', 'Material', 'Brand', 'Color']


In [ ]:
categorical_columns = df.select_dtypes(include=['object'])
cat_columns_with_nan = categorical_columns.columns[categorical_columns.isna().any()]
for column in sorted_columns:
    
    # Preparing the dataset
    df_with_target = df[df[column].notna()]
    df_with_nan_target = df[df[column].isna()]
        
    # Label Encoder
    le = LabelEncoder()
    df_with_target[column] = le.fit_transform(df_with_target[column])

    # Removing columns with NaN values
    df_with_target_without_nan_columns = df_with_target.dropna(axis=1, how='any')
    df_with_nan_target_without_nan_columns = df_with_nan_target.dropna(axis=1, how='any')
        
    # Checking for categorical values in df_with_nan_target_without_nan_columns
    columns_to_onehotencode = df_with_nan_target_without_nan_columns.select_dtypes(include='object').columns
    
    if len(columns_to_onehotencode) > 0:
        df_with_target_without_nan_columns = pd.get_dummies(df_with_target_without_nan_columns, columns= columns_to_onehotencode, drop_first=False)
        df_with_nan_target_without_nan_columns = pd.get_dummies(df_with_nan_target_without_nan_columns, columns=columns_to_onehotencode, drop_first=False)
    
    
    # Splitting features and targets
    X = df_with_target_without_nan_columns.drop(columns=column)
    y = df_with_target_without_nan_columns[column]
    
    # Model training
    model = RandomForestClassifier()
    model.fit(X, y)
    
    # Predciting missing values
    y_pred = model.predict(df_with_nan_target_without_nan_columns)
    
    # Replacing missing values in original dataframe by predicted values
    df.loc[df[column].isna(), column] = y_pred


C:\Users\julia\AppData\Local\Temp\ipykernel_20124\1969008462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])
C:\Users\julia\AppData\Local\Temp\ipykernel_20124\1969008462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


Entrou no if


C:\Users\julia\AppData\Local\Temp\ipykernel_20124\1969008462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


Entrou no if


C:\Users\julia\AppData\Local\Temp\ipykernel_20124\1969008462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


...

Entrou no if


C:\Users\julia\AppData\Local\Temp\ipykernel_20124\1969008462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


Entrou no if


C:\Users\julia\AppData\Local\Temp\ipykernel_20124\1969008462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_target[column] = le.fit_transform(df_with_target[column])


Entrou no if


In [161]:
df.isna().sum()

Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
dtype: int64

In [125]:
# Engenharia de atributos: get dummies para TODAS variáveis categóricas
df_eng = df.copy()
cat_columns = df_eng.select_dtypes(include=['object']).columns
for column in cat_columns:
    df_eng = pd.get_dummies(df_eng, columns=[column], prefix=['dum'], dtype=int)

scaler = MinMaxScaler()
df_eng['Weight Capacity (kg)'] = scaler.fit_transform(df_eng[['Weight Capacity (kg)']])
df_eng['Compartments'] = scaler.fit_transform(df_eng[['Compartments']])

In [126]:
X = df_eng.drop(columns='Price')
y = df_eng['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridge = Ridge(alpha=100)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

# Calculando o RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"{rmse:.5f}")

38.92012
